<a href="https://colab.research.google.com/github/pinilDissanayaka/GraphMind/blob/main/Experimental.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.1/208.1 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.7/407.7 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.5/113.5 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.5/294.5 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 872.8/872.8 kB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.6/296.6 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [16]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_groq.chat_models import ChatGroq
import requests
import os
from langchain_experimental.graph_transformers import LLMGraphTransformer

In [9]:
pdf_url="https://ec.europa.eu/echo/files/evaluation/watsan2005/annex_files/WEDC/diseases/cpd02.pdf"
pdf_name="cpd02.pdf"

response=requests.get(url=pdf_url)

with open(pdf_name,"wb") as f:
    f.write(response.content)

In [11]:
data=PyPDFLoader(file_path=pdf_name).load()

data[-1].page_content

'27DISEASE AND DISEASE TRANSMISSION\nThe period of communicability is the period in which the host is infectious, or the\nperiod in which pathogens are shed in the environment. The time between\nentrance of pathogen and the onset of communicability is the latent period. This is\nshown on a timeline in Figure 2.6.\nIn some infections the period of communicability starts before illness is apparent.\nHosts who can transmit the pathogen before showing symptoms are called incu-bating carriers. If the period of communicability extends beyond the end of the\nillness, the hosts are called convalescent carriers.'

In [12]:
splited_text=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=100).split_documents(documents=data)

In [15]:
from google.colab import userdata

os.environ["GROQ_API_KEY"]=userdata.get("GROQ_API_KEY")

llm=ChatGroq(model_name="llama-3.1-70b-versatile")

In [17]:
transformer=LLMGraphTransformer(llm=llm)

In [19]:
graph_documents=transformer.convert_to_graph_documents(documents=splited_text)

In [21]:
graph_documents[0]

GraphDocument(nodes=[Node(id='Pathogens', type='Infectious agent', properties={}), Node(id='Infectious Diseases', type='Disease', properties={}), Node(id='Infection', type='Disease', properties={}), Node(id='Infectious Agent', type='Organism', properties={}), Node(id='Disease Transmission Cycle', type='Cycle', properties={}), Node(id='Host', type='Person or animal', properties={}), Node(id='Environment', type='Environment', properties={})], relationships=[Relationship(source=Node(id='Pathogens', type='Infectious agent', properties={}), target=Node(id='Infectious Diseases', type='Disease', properties={}), type='CAUSE', properties={}), Relationship(source=Node(id='Pathogens', type='Infectious agent', properties={}), target=Node(id='Infection', type='Disease', properties={}), type='CAUSE', properties={}), Relationship(source=Node(id='Infectious Agent', type='Organism', properties={}), target=Node(id='Pathogens', type='Infectious agent', properties={}), type='IS_A', properties={}), Relatio